In [1]:
import json
import hashlib
from algosdk.v2client import algod
from algosdk import account, mnemonic
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn, wait_for_confirmation

Create Asset

In [2]:
algod_token = "2f3203f21e738a1de6110eba6984f9d03e5a95d7a577b34616854064cf2c0e7b"
algod_address = "https://academy-algod.dev.aws.algodev.network/"
algod_client = algod.AlgodClient(algod_token, algod_address)

In [7]:
def print_created_asset(algodclient, account, assetid):
  # note: if you have an indexer instance available it is easier to just use this
  # response = myindexer.accounts(asset_id = assetid)
  # then use 'account_info['created-assets'][0] to get info on the created asset
  account_info = algodclient.account_info(account)
  idx = 0
  for my_account_info in account_info['created-assets']:
    scrutinized_asset = account_info['created-assets'][idx]
    idx = idx + 1       
    if (scrutinized_asset['index'] == assetid):
      print("Asset ID: {}".format(scrutinized_asset['index']))
      print(json.dumps(my_account_info['params'], indent=4))
      break

#   Utility function used to print asset holding for account and assetid
def print_asset_holding(algodclient, account, assetid):
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then loop thru the accounts returned and match the account you are looking for
    account_info = algodclient.account_info(account)
    idx = 0
    for my_account_info in account_info['assets']:
        scrutinized_asset = account_info['assets'][idx]
        idx = idx + 1        
        if (scrutinized_asset['asset-id'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['asset-id']))
            print(json.dumps(scrutinized_asset, indent=4))
            break

In [21]:
# accepts the mnemonic
def create_asset(m,receiver_pk):  
  accounts = {}
  accounts[1] = {}
  accounts[1]['pk'] = mnemonic.to_public_key(m)
  accounts[1]['sk'] = mnemonic.to_private_key(m)

  print("--------------------------------------------")
  print("Creating Asset...")
  # CREATE ASSET
  # Get network params for transactions before every transaction.
  params = algod_client.suggested_params()
  # comment these two lines if you want to use suggested params
  # params.fee = 1000
  # params.flat_fee = True
    
  # JSON file
#   dir_path = os.path.dirname(os.path.realpath(__file__))
#   f = open (dir_path + '/NFT/metadata.json', "r")
  f = open ('../NFT_py/NFT/metadata.json', "r")


  
  # Reading from file
  metadataJSON = json.loads(f.read())
  metadataStr = json.dumps(metadataJSON)

  hash = hashlib.new("sha512_256")
  hash.update(b"arc0003/amj")
  hash.update(metadataStr.encode("utf-8"))
  json_metadata_hash = hash.digest()


  # Account 1 creates an asset called latinum and
  # sets Account 1 as the manager, reserve, freeze, and clawback address.
  # Asset Creation transaction
  txn = AssetConfigTxn(
      sender=accounts[1]['pk'],
      sp=params,
      total=1,
      default_frozen=False,
      unit_name="ALICE001",
      asset_name="Alice's Artwork 001",
      manager=accounts[1]['pk'],
      reserve=receiver_pk,
      freeze=None,
      clawback=None,
      strict_empty_address_check=False,
      url="https://path/to/my/asset/details", 
      metadata_hash=json_metadata_hash,
      decimals=0)

  # Sign with secret key of creator
  stxn = txn.sign(accounts[1]['sk'])

  # Send the transaction to the network and retrieve the txid.
  txid = algod_client.send_transaction(stxn)
  print("Asset Creation Transaction ID: {}".format(txid))


  # Wait for the transaction to be confirmed
  confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
  print("TXID: ", txid)
  print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
  try:
      # Pull account info for the creator
      # account_info = algod_client.account_info(accounts[1]['pk'])
      # get asset_id from tx
      # Get the new asset's information from the creator account
      ptx = algod_client.pending_transaction_info(txid)
      asset_id = ptx["asset-index"]
      print_created_asset(algod_client, accounts[1]['pk'], asset_id)
      print_asset_holding(algod_client, accounts[1]['pk'], asset_id)
  except Exception as e:
      print(e)


In [3]:
# OPT-IN
def opt_in(trainee_m,asset_id):

    # Check if asset_id is in account 3's asset holdings prior
    # to opt-in
    trainee_sk=mnemonic.to_private_key(trainee_m)
    trainee_pk=mnemonic.to_public_key(trainee_m)
    params = algod_client.suggested_params()
    # comment these two lines if you want to use suggested params
    params.fee = 1000
    params.flat_fee = True


    # Use the AssetTransferTxn class to transfer assets and opt-in
    txn = AssetTransferTxn(
        sender=trainee_pk,
        sp=params,
        receiver=trainee_pk,
        amt=0,
        index=asset_id)
    stxn = txn.sign(trainee_sk)
    txid = algod_client.send_transaction(stxn)
    print(txid)
    # Wait for the transaction to be confirmed
    wait_for_confirmation(algod_client, txid)
    # Now check the asset holding for that account.
    # This should now show a holding with a balance of 0.
    print_asset_holding(algod_client, trainee_pk, asset_id)

In [6]:
# TRANSFER ASSET
def transfer_asset(sender_m,receiver_pk,asset_id):

    sender_pk=mnemonic.to_public_key(sender_m)
    sender_sk=mnemonic.to_private_key(sender_m)

    # transfer asset of 10 from account 1 to account 3
    params = algod_client.suggested_params()
    # comment these two lines if you want to use suggested params
    params.fee = 1000
    params.flat_fee = True
    txn = AssetTransferTxn(
        sender=sender_pk,
        sp=params,
        receiver=receiver_pk,
        amt=1,
        index=asset_id)
    stxn = txn.sign(sender_sk)
    txid = algod_client.send_transaction(stxn)
    print(txid)
    # Wait for the transaction to be confirmed
    wait_for_confirmation(algod_client, txid)
    # The balance should now be 10.
    print_asset_holding(algod_client, receiver_pk, asset_id)

## sender

In [18]:
import os,sys
cwd = os.getcwd()
sys.path.append(f'../NFT_py/')
from create_account import create_account
m=create_account()

My address: EUVBE6MISEX3QLZYERRCPJBJYXXSXM3GQU5DY6EEO7VGAHBFFEZLMXVET4
Account balance: 0 microAlgos

Go to the below link to fund the created account using testnet faucet: 
 https://dispenser.testnet.aws.algodev.network/?account=EUVBE6MISEX3QLZYERRCPJBJYXXSXM3GQU5DY6EEO7VGAHBFFEZLMXVET4
Fund transfer in process...
Fund transferred!
Account balance: 5000000 microAlgos



## receiver

In [20]:
receiver=create_account()

My address: JLPXYTO2Z5KJWBG2HJLJHJYCRUNDU2AH6QQOYWA6G6CCGN6NJYSBR62CGA
Account balance: 0 microAlgos

Go to the below link to fund the created account using testnet faucet: 
 https://dispenser.testnet.aws.algodev.network/?account=JLPXYTO2Z5KJWBG2HJLJHJYCRUNDU2AH6QQOYWA6G6CCGN6NJYSBR62CGA
Fund transfer in process...
Fund transferred!
Account balance: 5000000 microAlgos



In [22]:
receiver_pk="JLPXYTO2Z5KJWBG2HJLJHJYCRUNDU2AH6QQOYWA6G6CCGN6NJYSBR62CGA"

In [23]:
create_asset(m,receiver_pk)

--------------------------------------------
Creating Asset...
Asset Creation Transaction ID: MB36A2KG2ZHD3K2OAEGF7AOT6RGO2UZJUORFFPHPIIC66CF57YJQ
TXID:  MB36A2KG2ZHD3K2OAEGF7AOT6RGO2UZJUORFFPHPIIC66CF57YJQ
Result confirmed in round: 24462866
Asset ID: 113735460
{
    "creator": "EUVBE6MISEX3QLZYERRCPJBJYXXSXM3GQU5DY6EEO7VGAHBFFEZLMXVET4",
    "decimals": 0,
    "default-frozen": false,
    "manager": "EUVBE6MISEX3QLZYERRCPJBJYXXSXM3GQU5DY6EEO7VGAHBFFEZLMXVET4",
    "metadata-hash": "5zAUYEQ+kEKv8VCUNu17DMVNzpzRrF7X9JUr+OEfgp4=",
    "name": "Alice's Artwork 001",
    "name-b64": "QWxpY2UncyBBcnR3b3JrIDAwMQ==",
    "reserve": "JLPXYTO2Z5KJWBG2HJLJHJYCRUNDU2AH6QQOYWA6G6CCGN6NJYSBR62CGA",
    "total": 1,
    "unit-name": "ALICE001",
    "unit-name-b64": "QUxJQ0UwMDE=",
    "url": "https://path/to/my/asset/details",
    "url-b64": "aHR0cHM6Ly9wYXRoL3RvL215L2Fzc2V0L2RldGFpbHM="
}
Asset ID: 113735460
{
    "amount": 1,
    "asset-id": 113735460,
    "is-frozen": false
}


In [29]:
opt_in(receiver,asset_id)

J376TYWKEZXEXOPPF2GMAIYAVI6A2FDBLKCI67SWSWC46Z2EDYNQ
Asset ID: 113735460
{
    "amount": 0,
    "asset-id": 113735460,
    "is-frozen": false
}


In [33]:
transfer_asset(m,receiver_pk,asset_id)

O4I6I3MADSBVJLGPGVI7BI2A6CJTKMPYZELEYXNWP4RIIVLQRNCQ
Asset ID: 113735460
{
    "amount": 1,
    "asset-id": 113735460,
    "is-frozen": false
}


In [34]:
create_asset(m,None)

--------------------------------------------
Creating Asset...
Asset Creation Transaction ID: QCF2NXOUUA2QEQDUGI2FCOOG4ECAZE3ZJ2XJM5IGF2O7VHYVHTQQ
TXID:  QCF2NXOUUA2QEQDUGI2FCOOG4ECAZE3ZJ2XJM5IGF2O7VHYVHTQQ
Result confirmed in round: 24463074
Asset ID: 113736954
{
    "creator": "EUVBE6MISEX3QLZYERRCPJBJYXXSXM3GQU5DY6EEO7VGAHBFFEZLMXVET4",
    "decimals": 0,
    "default-frozen": false,
    "manager": "EUVBE6MISEX3QLZYERRCPJBJYXXSXM3GQU5DY6EEO7VGAHBFFEZLMXVET4",
    "metadata-hash": "5zAUYEQ+kEKv8VCUNu17DMVNzpzRrF7X9JUr+OEfgp4=",
    "name": "Alice's Artwork 001",
    "name-b64": "QWxpY2UncyBBcnR3b3JrIDAwMQ==",
    "total": 1,
    "unit-name": "ALICE001",
    "unit-name-b64": "QUxJQ0UwMDE=",
    "url": "https://path/to/my/asset/details",
    "url-b64": "aHR0cHM6Ly9wYXRoL3RvL215L2Fzc2V0L2RldGFpbHM="
}
Asset ID: 113736954
{
    "amount": 1,
    "asset-id": 113736954,
    "is-frozen": false
}


## transferring asset without optin results in error

In [35]:
asset_id2=113736954
transfer_asset(m,receiver_pk,asset_id2)

AlgodHTTPError: TransactionPool.Remember: transaction 2ATSOIPKG2PPRTQJ5EVCC7C2IDY5QRGLH3ALHRBBLUN3K5TFVPBQ: receiver error: must optin, asset 113736954 missing from JLPXYTO2Z5KJWBG2HJLJHJYCRUNDU2AH6QQOYWA6G6CCGN6NJYSBR62CGA

Get asset id

In [36]:
create_asset(m,receiver_pk)

--------------------------------------------
Creating Asset...
Asset Creation Transaction ID: MYRFXKFV3R63NUEG33BEP7VLJ26W7EKANKSYPW5O6COSROOPOE5Q
TXID:  MYRFXKFV3R63NUEG33BEP7VLJ26W7EKANKSYPW5O6COSROOPOE5Q
Result confirmed in round: 24463182
Asset ID: 113737741
{
    "creator": "EUVBE6MISEX3QLZYERRCPJBJYXXSXM3GQU5DY6EEO7VGAHBFFEZLMXVET4",
    "decimals": 0,
    "default-frozen": false,
    "manager": "EUVBE6MISEX3QLZYERRCPJBJYXXSXM3GQU5DY6EEO7VGAHBFFEZLMXVET4",
    "metadata-hash": "5zAUYEQ+kEKv8VCUNu17DMVNzpzRrF7X9JUr+OEfgp4=",
    "name": "Alice's Artwork 001",
    "name-b64": "QWxpY2UncyBBcnR3b3JrIDAwMQ==",
    "reserve": "JLPXYTO2Z5KJWBG2HJLJHJYCRUNDU2AH6QQOYWA6G6CCGN6NJYSBR62CGA",
    "total": 1,
    "unit-name": "ALICE001",
    "unit-name-b64": "QUxJQ0UwMDE=",
    "url": "https://path/to/my/asset/details",
    "url-b64": "aHR0cHM6Ly9wYXRoL3RvL215L2Fzc2V0L2RldGFpbHM="
}
Asset ID: 113737741
{
    "amount": 1,
    "asset-id": 113737741,
    "is-frozen": false
}


In [37]:
account_info = algod_client.account_info(receiver_pk)
# holding = None
# idx = 0
# for my_account_info in account_info['assets']:
#     scrutinized_asset = account_info['assets'][idx]
#     idx = idx + 1    
#     if (scrutinized_asset['asset-id'] == asset_id):
#         holding = True
#         break
account_info

{'address': 'JLPXYTO2Z5KJWBG2HJLJHJYCRUNDU2AH6QQOYWA6G6CCGN6NJYSBR62CGA',
 'amount': 4999000,
 'amount-without-pending-rewards': 4999000,
 'apps-local-state': [],
 'apps-total-schema': {'num-byte-slice': 0, 'num-uint': 0},
 'assets': [{'amount': 1, 'asset-id': 113735460, 'is-frozen': False}],
 'created-apps': [],
 'created-assets': [],
 'min-balance': 200000,
 'pending-rewards': 0,
 'reward-base': 27521,
 'rewards': 0,
 'round': 24463192,
 'status': 'Offline',
 'total-apps-opted-in': 0,
 'total-assets-opted-in': 1,
 'total-created-apps': 0,
 'total-created-assets': 0}